# PaddleNLP词向量应用展示

6.7日NLP直播打卡课开始啦

**[直播链接请戳这里，每晚20:00-21:30👈](http://live.bilibili.com/21689802)**

**[课程地址请戳这里👈](https://aistudio.baidu.com/aistudio/course/introduce/24177)**

欢迎来课程**QQ群**（群号:618354318）交流吧~~


词向量（Word embedding），即把词语表示成实数向量。“好”的词向量能体现词语直接的相近关系。词向量已经被证明可以提高NLP任务的性能，例如语法分析和情感分析。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/54878855b1df42f9ab50b280d76906b1e0175f280b0f4a2193a542c72634a9bf" width="60%" height="50%"> <br />
</p>
<br><center>图1：词向量示意图</center></br>

PaddleNLP已预置多个公开的预训练Embedding，您可以通过使用`paddlenlp.embeddings.TokenEmbedding`接口加载各种预训练Embedding。本篇教程将介绍`paddlenlp.embeddings.TokenEmbedding`的使用方法，计算词与词之间的语义距离，并结合词袋模型获取句子的语义表示。

In [ ]:
!pip install --upgrade paddlenlp -i https://pypi.org/simple

     |████████████████████████████████| 430kB 20kB/s eta 0:00:013
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1


## 加载TokenEmbedding

`TokenEmbedding()`参数
- `embedding_name`
将模型名称以参数形式传入TokenEmbedding，加载对应的模型。默认为`w2v.baidu_encyclopedia.target.word-word.dim300`的词向量。
- `unknown_token`
未知token的表示，默认为[UNK]。
- `unknown_token_vector`
未知token的向量表示，默认生成和embedding维数一致，数值均值为0的正态分布向量。
- `extended_vocab_path`
扩展词汇列表文件路径，词表格式为一行一个词。如引入扩展词汇列表，trainable=True。
- `trainable`
Embedding层是否可被训练。True表示Embedding可以更新参数，False为不可更新。默认为True。

In [36]:
from paddlenlp.embeddings import TokenEmbedding

# 初始化TokenEmbedding， 预训练embedding未下载时会自动下载并加载数据
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300") # 百度百科
token_embedding1 = TokenEmbedding(embedding_name="w2v.wiki.target.word-word.dim300") # 中文维基百科
token_embedding2 = TokenEmbedding(embedding_name="w2v.sogou.target.word-word.dim300") # 搜狗新闻
# 查看token_embedding详情
print(token_embedding)
print(token_embedding1)
print(token_embedding2)

[2021-06-08 23:43:59,922] [    INFO] - Loading token embedding...
[2021-06-08 23:44:11,922] [    INFO] - Finish loading embedding vector.
[2021-06-08 23:44:11,925] [    INFO] - Token Embedding info:             
Unknown index: 635963             
Unknown token: [UNK]             
Padding index: 635964             
Padding token: [PAD]             
Shape :[635965, 300]
[2021-06-08 23:44:12,112] [    INFO] - Loading token embedding...
[2021-06-08 23:44:19,050] [    INFO] - Finish loading embedding vector.
[2021-06-08 23:44:19,053] [    INFO] - Token Embedding info:             
Unknown index: 352217             
Unknown token: [UNK]             
Padding index: 352218             
Padding token: [PAD]             
Shape :[352219, 300]
[2021-06-08 23:44:19,135] [    INFO] - Loading token embedding...
[2021-06-08 23:44:25,912] [    INFO] - Finish loading embedding vector.
[2021-06-08 23:44:25,914] [    INFO] - Token Embedding info:             
Unknown index: 364990             
Unknown tok

Object   type: TokenEmbedding(635965, 300, padding_idx=635964, sparse=False)             
Unknown index: 635963             
Unknown token: [UNK]             
Padding index: 635964             
Padding token: [PAD]             
Parameter containing:
Tensor(shape=[635965, 300], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[-0.24200200,  0.13931701,  0.07378800, ...,  0.14103900,  0.05592300, -0.08004800],
        [-0.08671700,  0.07770800,  0.09515300, ...,  0.11196400,  0.03082200, -0.12893000],
        [-0.11436500,  0.12201900,  0.02833000, ...,  0.11068700,  0.03607300, -0.13763499],
        ...,
        [ 0.02628800, -0.00008300, -0.00393500, ...,  0.00654000,  0.00024600, -0.00662600],
        [-0.02026969, -0.04112981,  0.01609680, ...,  0.04630239,  0.02837262, -0.02780960],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,  0.        ,  0.        ]])
Object   type: TokenEmbedding(352219, 300, padding_idx=352218, sparse=False)             
Unknown 

### 认识一下Embedding
**`TokenEmbedding.search()`**
获得指定词汇的词向量。

In [ ]:
# 没去看源码，但是可推测TokenEmbedding.search()参数可以是列表
print(token_embedding.search(["列表","字符串"]))
print(token_embedding.search("列表","字符串"))

[[ 1.086150e-01  2.681490e-01  2.231060e-01  1.712660e-01  6.545800e-02
   3.844280e-01  3.155260e-01 -1.412230e-01 -1.970900e-01 -5.488100e-02
  -1.620380e-01  1.107060e-01 -2.075200e-01 -2.039100e-01 -2.166950e-01
   9.570100e-02  5.213390e-01  6.832540e-01  3.410500e-01 -1.630890e-01
  -2.013300e-02 -3.416260e-01 -1.619600e-02 -1.484080e-01 -6.142130e-01
  -3.336550e-01 -5.280800e-01  7.788900e-02  2.677860e-01 -1.304590e-01
  -1.420620e-01  7.710000e-04  2.866000e-01  6.736000e-03  2.218710e-01
  -2.267790e-01 -2.923430e-01  7.641200e-02  4.093050e-01 -2.702300e-02
  -1.443730e-01 -1.656810e-01 -3.560840e-01 -5.357700e-02  1.814660e-01
  -2.028670e-01 -3.502670e-01 -4.418250e-01 -1.467600e-02 -8.587330e-01
   2.473920e-01  2.158060e-01  5.702700e-02  7.885600e-02 -4.507010e-01
   1.301820e-01 -2.273600e-02 -6.316400e-02 -2.883230e-01 -3.469960e-01
   1.877860e-01 -4.591000e-02 -1.589500e-01 -2.843330e-01  1.985380e-01
   2.436840e-01  1.602500e-01 -4.388910e-01 -3.791320e-01  6.029

TypeError: search() takes 2 positional arguments but 3 were given

In [ ]:
test_token_embedding = token_embedding.search("中国")
test_token_embedding1 = token_embedding.search("百度")
print(test_token_embedding)
print(test_token_embedding1)

[[ 0.260801  0.1047    0.129453 -0.257317 -0.16152   0.19567  -0.074868
   0.361168  0.245882 -0.219141 -0.388083  0.235189  0.029316  0.154215
  -0.354343  0.017746  0.009028  0.01197  -0.121429  0.096542  0.009255
   0.039721  0.363704 -0.239497 -0.41168   0.16958   0.261758  0.022383
  -0.053248 -0.000994 -0.209913 -0.208296  0.197332 -0.3426   -0.162112
   0.134557 -0.250201  0.431298  0.303116  0.517221  0.243843  0.022219
  -0.136554 -0.189223  0.148563 -0.042963 -0.456198  0.14546  -0.041207
   0.049685  0.20294   0.147355 -0.206953 -0.302796 -0.111834  0.128183
   0.289539 -0.298934 -0.096412  0.063079  0.324821 -0.144471  0.052456
   0.088761 -0.040925 -0.103281 -0.216065 -0.200878 -0.100664  0.170614
  -0.355546 -0.062115 -0.52595  -0.235442  0.300866 -0.521523 -0.070713
  -0.331768  0.023021  0.309111 -0.125696  0.016723 -0.0321   -0.200611
   0.057294 -0.128891 -0.392886  0.423002  0.282569 -0.212836  0.450132
   0.067604 -0.124928 -0.294086  0.136479  0.091505 -0.061723 -0

**`TokenEmbedding.cosine_sim()`**
计算词向量间余弦相似度，语义相近的词语余弦相似度更高，说明预训练好的词向量空间有很好的语义表示能力。

In [39]:
score1 = token_embedding.cosine_sim("女孩", "女人")
score2 = token_embedding.cosine_sim("女孩", "书籍")
score3 = token_embedding.cosine_sim("男孩", "书籍")

score4 = token_embedding1.cosine_sim("女孩", "书籍")
score5 = token_embedding1.cosine_sim("男孩", "书籍")

score6 = token_embedding2.cosine_sim("女孩", "书籍")
score7 = token_embedding2.cosine_sim("男孩", "书籍")
print('score1:', score1)
print('score2:', score2)
print('score3:', score3)
print('score4:', score4)
print('score5:', score5)
print('score6:', score6)
print('score7:', score7)

score1: 0.7017183
score2: 0.19189896
score3: 0.17004019
score4: 0.20850094
score5: 0.19129024
score6: 0.1923712
score7: 0.1871469


In [37]:
scorea = token_embedding.cosine_sim("男", "护士")
scoreb = token_embedding.cosine_sim("女", "护士")
print(scorea, scoreb)
scorea1 = token_embedding1.cosine_sim("男", "护士")
scoreb1 = token_embedding1.cosine_sim("女", "护士")
print(scorea1, scoreb1)
scorea2 = token_embedding2.cosine_sim("男", "护士")
scoreb2 = token_embedding2.cosine_sim("女", "护士")
print(scorea2, scoreb2)

0.21026553 0.28488365
0.20323813 0.26153284
0.27946827 0.265471


### 词向量映射到低维空间

使用深度学习可视化工具[VisualDL](https://github.com/PaddlePaddle/VisualDL)的[High Dimensional](https://github.com/PaddlePaddle/VisualDL/blob/develop/docs/components/README_CN.md#High-Dimensional--%E6%95%B0%E6%8D%AE%E9%99%8D%E7%BB%B4%E7%BB%84%E4%BB%B6)组件可以对embedding结果进行可视化展示，便于对其直观分析，步骤如下：

1. 升级 VisualDL 最新版本。

`pip install --upgrade visualdl`

2. 创建LogWriter并将记录词向量。

3. 点击左侧面板中的可视化tab，选择‘token_hidi’作为文件并启动VisualDL可视化

In [ ]:
!pip install --upgrade visualdl

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 2.7MB 21.2MB/s eta 0:00:01
  Found existing installation: visualdl 2.1.1
    Uninstalling visualdl-2.1.1:
      Successfully uninstalled visualdl-2.1.1


In [ ]:
print(token_embedding.vocab.to_tokens(list(range(1,10))))
print(token_embedding.vocab.to_tokens(list(range(-10,-1))))

['的', '。', '、', ',', '：', '和', '在', '了', '是']
['马善钧', '徐斌富', 'Appearson', 'yeletmebewithyouwoo', "it'sonlylove", 'ToyWatch', 'LilFlip', '欧仕益', '[UNK]']


In [40]:
# 获取词表中前1000个单词

labels = token_embedding.vocab.to_tokens(list(range(0, 1000)))
# 取出这1000个单词对应的Embedding
test_token_embedding = token_embedding.search(labels)
test_token_embedding1 = token_embedding1.search(labels)
test_token_embedding2 = token_embedding2.search(labels)

# 引入VisualDL的LogWriter记录日志
from visualdl import LogWriter

with LogWriter(logdir='./token_hidi') as writer:
    writer.add_embeddings(tag='test', mat=[i for i in test_token_embedding], metadata=labels)
    writer.add_embeddings(tag='test1', mat=[i for i in test_token_embedding1], metadata=labels)
    writer.add_embeddings(tag='test2', mat=[i for i in test_token_embedding2], metadata=labels)

#### 启动VisualDL查看词向量降维效果
启动步骤：
- 1、切换到「可视化」指定可视化日志
- 2、日志文件选择 'token_hidi'
- 3、点击「启动VisualDL」后点击「打开VisualDL」，选择「高维数据映射」，即可查看词表中前1000词UMAP方法下映射到三维空间的可视化结果:

![](https://user-images.githubusercontent.com/48054808/120594172-1fe02b00-c473-11eb-9df1-c0206b07e948.gif)

可以看出，语义相近的词在词向量空间中聚集(如数字、章节等)，说明预训练好的词向量有很好的文本表示能力。

使用VisualDL除可视化embedding结果外，还可以对标量、图片、音频等进行可视化，有效提升训练调参效率。关于VisualDL更多功能和详细介绍，可参考[VisualDL使用文档](https://github.com/PaddlePaddle/VisualDL/tree/develop/docs)。

## 基于TokenEmbedding衡量句子语义相似度

在许多实际应用场景（如文档检索系统）中， 需要衡量两个句子的语义相似程度。此时我们可以使用词袋模型（Bag of Words，简称BoW）计算句子的语义向量。

**首先**，将两个句子分别进行切词，并在TokenEmbedding中查找相应的单词词向量（word embdding）。

**然后**，根据词袋模型，将句子的word embedding叠加作为句子向量（sentence embedding）。

**最后**，计算两个句子向量的余弦相似度。

### 基于TokenEmbedding的词袋模型


使用`BoWEncoder`搭建一个BoW模型用于计算句子语义。

* `paddlenlp.TokenEmbedding`组建word-embedding层
* `paddlenlp.seq2vec.BoWEncoder`组建句子建模层


In [ ]:
import paddle
import paddle.nn as nn
import paddlenlp


class BoWModel(nn.Layer):
    def __init__(self, embedder):
        super().__init__()
        self.embedder = embedder
        emb_dim = self.embedder.embedding_dim
        self.encoder = paddlenlp.seq2vec.BoWEncoder(emb_dim)
        self.cos_sim_func = nn.CosineSimilarity(axis=-1)

    def get_cos_sim(self, text_a, text_b):
        text_a_embedding = self.forward(text_a)
        text_b_embedding = self.forward(text_b)
        cos_sim = self.cos_sim_func(text_a_embedding, text_b_embedding)
        return cos_sim

    def forward(self, text):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)

        # Shape: (batch_size, embedding_dim)
        summed = self.encoder(embedded_text)

        return summed

model = BoWModel(embedder=token_embedding)

### 构造Tokenizer
使用TokenEmbedding词表构造Tokenizer。

In [41]:
from data import Tokenizer
tokenizer = Tokenizer()
tokenizer1 = Tokenizer()
tokenizer2 = Tokenizer()
tokenizer.set_vocab(vocab=token_embedding.vocab)
tokenizer1.set_vocab(vocab=token_embedding1.vocab)
tokenizer2.set_vocab(vocab=token_embedding2.vocab)

### 相似句对数据读取

以提供的样例数据text_pair.txt为例，该数据文件每行包含两个句子。


In [ ]:
text_pairs = {}
with open("text_pair.txt", "r", encoding="utf8") as f:
    for line in f:
        text_a, text_b = line.strip().split("\t")
        if text_a not in text_pairs:
            text_pairs[text_a] = []
        text_pairs[text_a].append(text_b)

### 查看相似语句相关度

In [42]:
for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])
    text_a_ids1 = paddle.to_tensor([tokenizer1.text_to_ids(text_a)])
    text_a_ids2 = paddle.to_tensor([tokenizer2.text_to_ids(text_a)])

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        text_b_ids1 = paddle.to_tensor([tokenizer1.text_to_ids(text_b)])
        text_b_ids2 = paddle.to_tensor([tokenizer2.text_to_ids(text_b)])
        print("text_a: {}".format(text_a))
        print("text_b: {}".format(text_b))
        print("cosine_sim: {}".format(model.get_cos_sim(text_a_ids, text_b_ids).numpy()[0]))
        print("cosine_sim1: {}".format(model.get_cos_sim(text_a_ids1, text_b_ids1).numpy()[0]))
        print("cosine_sim2: {}".format(model.get_cos_sim(text_a_ids2, text_b_ids2).numpy()[0]))
        print()

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 多项式矩阵的左共轭积及其应用
cosine_sim: 0.8861939311027527
cosine_sim1: 0.8283802270889282
cosine_sim2: 0.8167858123779297

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 退化阻尼对高维可压缩欧拉方程组经典解的影响
cosine_sim: 0.7975841760635376
cosine_sim1: 0.7183640003204346
cosine_sim2: 0.7050017714500427

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: Burgers方程基于特征正交分解方法的数值解法研究
cosine_sim: 0.818878173828125
cosine_sim1: 0.7362374067306519
cosine_sim2: 0.7573085427284241

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 有界对称域上解析函数空间的若干性质
cosine_sim: 0.8041475415229797
cosine_sim1: 0.6916273832321167
cosine_sim2: 0.648672878742218

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于卷积神经网络的图像复杂度研究与应用
cosine_sim: 0.7444741129875183
cosine_sim1: 0.7012829184532166
cosine_sim2: 0.7238141894340515

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: Cartesian发射机中线性功率放大器的研究
cosine_sim: 0.7536823749542236
cosine_sim1: 0.7144468426704407
cosine_sim2: 0.7386630773544312

text_a

### 使用VisualDL查看句子向量

In [44]:
# 引入VisualDL的LogWriter记录日志
import numpy as np
from visualdl import LogWriter    
# 获取句子以及其对应的向量
label_list = []
embedding_list = []
embedding_list1 = []
embedding_list2 = []

for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])
    text_a_ids1 = paddle.to_tensor([tokenizer1.text_to_ids(text_a)])
    text_a_ids2 = paddle.to_tensor([tokenizer2.text_to_ids(text_a)])
    embedding_list.append(model(text_a_ids).flatten().numpy())
    embedding_list1.append(model(text_a_ids1).flatten().numpy())
    embedding_list2.append(model(text_a_ids2).flatten().numpy())
    label_list.append(text_a)

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        text_b_ids1 = paddle.to_tensor([tokenizer1.text_to_ids(text_b)])
        text_b_ids2 = paddle.to_tensor([tokenizer2.text_to_ids(text_b)])
        embedding_list.append(model(text_b_ids).flatten().numpy())
        embedding_list1.append(model(text_b_ids1).flatten().numpy())
        embedding_list2.append(model(text_b_ids2).flatten().numpy())
        label_list.append(text_b)


with LogWriter(logdir='./sentence_hidi') as writer:
    writer.add_embeddings(tag='test', mat=embedding_list, metadata=label_list)
    writer.add_embeddings(tag='test1', mat=embedding_list1, metadata=label_list)
    writer.add_embeddings(tag='test2', mat=embedding_list2, metadata=label_list)

### 启动VisualDL观察句子向量降维效果

步骤如上述观察词向量降维效果一模一样。
![](https://ai-studio-static-online.cdn.bcebos.com/0e876f3cf1724e90a317ad3f4be233a9eb0313b0e92f475b95675c2ad52d3eb0)


可以看出，语义相近的句子在句子向量空间中聚集(如有关课堂的句子、有关化学描述句子等)。

## 作业

更换TokenEmbedding预训练模型，使用VisualDL查看相应的TokenEmbedding可视化效果，并尝试更换后的TokenEmbedding计算句对语义相似度。
本作业详细步骤，可参考[Day01作业教程](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/education/day01.md)，记得star PaddleNLP，收藏起来，随时跟进最新功能噢。

**作业结果提交**：
1. 截图提交可视化结果（图片注明作业可视化结果）。
2. 通篇执行每段代码，并保留执行结果。

#### 词向量可视化结果

##### 百度百科
![百度百科](https://ai-studio-static-online.cdn.bcebos.com/fb9ca0e219864b8eb1ca6c6bebe02188683b8c57909d4179ad61cbc4dd923160)

##### 中文维基
![中文维基](https://ai-studio-static-online.cdn.bcebos.com/44e9e14e93ea47269342f93eef886293e59da9e0af53422ca60ae658406a5555)

##### 搜狗新闻
![搜狗新闻](https://ai-studio-static-online.cdn.bcebos.com/897e87628e2d4de39e8de8a8c47f9bf106875136b59c4682aa87e4f0b2664cf8)


#### 句向量可视化结果

##### 百度百科
![百度百科](https://ai-studio-static-online.cdn.bcebos.com/3032c79dcf6a4265b44d936e51984a46075095fae722417ea5972accea5de2f8)

##### 中文维基
![中文维基](https://ai-studio-static-online.cdn.bcebos.com/2d97ccc1c315460eb3a92ab16406be6ad95c7b47059e4e46b1a2caa54ebfef04)

##### 搜狗新闻
![搜狗新闻](https://ai-studio-static-online.cdn.bcebos.com/5f8e559787304ce49d20c18b7ea890c77fb2bbeb802142f1981ea1cb8f40bb49)



# PaddleNLP更多预训练词向量
PaddleNLP提供61种可直接加载的预训练词向量，训练自多领域中英文语料、如百度百科、新闻语料、微博等，覆盖多种经典词向量模型（word2vec、glove、fastText）、涵盖不同维度、不同语料库大小，详见[PaddleNLP Embedding API](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/model_zoo/embeddings.md)。

# 预训练词向量辅助分类任务

想学习词向量更多应用，来试试预训练词向量对分类模型的改善效果吧，[这里](https://aistudio.baidu.com/aistudio/projectdetail/1283423) 试试把`paddle.nn.Embedding`换成刚刚学到的预训练词向量吧。

# 加入课程交流群，一起学习吧

现在就加入课程群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >



**[直播链接请戳这里，每晚20:00-21:30👈](http://live.bilibili.com/21689802)**

**[还没有报名课程？赶紧戳这里，课程、作业安排统统在课程区哦👉🏻](https://aistudio.baidu.com/aistudio/course/introduce/24177)**